In [ ]:
import cv2
import numpy as np 


def segment_lines(lines, delta):
    h_lines = []
    v_lines = []
    for line in lines:
        for x1, y1, x2, y2 in line:
            if abs(x2-x1) < delta: # x-values are near; line is vertical
                v_lines.append(line)
            elif abs(y2-y1) < delta: # y-values are near; line is horizontal
                h_lines.append(line)
    return h_lines, v_lines


def main():
    
    windowName = "HoughPLineDetector"
    cv2.namedWindow(windowName)
    cap = cv2.VideoCapture(0)
    
    if cap.isOpened():
        ret, frame = cap.read()
    else:
        ret = False


    while ret:
    
        ret, frame = cap.read() 

        # preprocessing
        frame = cv2.GaussianBlur(frame, (7, 7), 1.41) # put camera through gaussian filter to blur it
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray, 50, 150)
        dilated = cv2.dilate(edges, np.ones((3,3), dtype=np.uint8))


        # run the Hough transform
        lines = cv2.HoughLinesP(dilated, rho=1, theta=np.pi/180, threshold=50, maxLineGap=0, minLineLength=50)

        # segment the lines
        delta = 10
        h_lines, v_lines = segment_lines(lines, delta)

        # draw the segmented lines
        for line in h_lines:
            for x1, y1, x2, y2 in line:
                color = [0,0,255] # color hoz lines red
                cv2.line(frame, (x1, y1), (x2, y2), color=color, thickness=2)
        for line in v_lines:
            for x1, y1, x2, y2 in line:
                color = [255,0,0] # color vert lines blue
                cv2.line(frame, (x1, y1), (x2, y2), color=color, thickness=2)



        cv2.imshow("Center of intersection clusters", frame)
        if cv2.waitKey(1) == 27: # exit on ESC
            break

    cv2.destroyAllWindows()
    cap.release()

if __name__ == "__main__":
    main()
